### Baseline model (Plus)에 있는 기능

- binary_load_dataset 메서드

데이터 비율 조정을 위한 파라미터 값을 기반으로 데이터를 로드하는 메서드로

주어진 binary_classification_data.csv 파일을 열고 데이터를 읽습니다. 데이터의 마지막 열에 종속 변수가 있으며, 해당 열의 값에 따라서 '1'인 pulsar 데이터와 '0'인 star 데이터를 각각 별도의 리스트에 저장합니다.

그 후, 실험에 사용될 데이터와 신경망의 입력 및 출력 크기를 전역 변수로 선언합니다. pulsar와 star 데이터는 하나의 변수 data에 그룹화됩니다. pulsar 데이터의 수를 star 데이터의 수에 맞추기 위해 데이터의 수를 조정할 수 있습니다.

adjust_ratio 매개변수가 True인 경우, pulsar 데이터의 비율을 증가시킵니다. pulsar 데이터와 star 데이터를 하나의 변수에 할당하고, 데이터의 크기는 star 데이터의 배수로 저장됩니다.

adjust_ratio 값이 True인 경우 star 데이터를 data 변수에 할당한 후, pulsar 데이터를 star 데이터의 수만큼 반복하여 추가합니다. 이를 위해 % 연산자를 사용하여 pulsar 데이터의 인덱스를 순차적으로 추출합니다.

adjust_ratio 값이 False인 경우, 기존 데이터를 그대로 data 변수에 할당합니다. star 데이터를 먼저 할당하고, 그 다음에 pulsar 데이터를 할당합니다.

### 데이터 불러오기

In [22]:
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
from keras.layers import Dropout
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

In [2]:
df = pd.read_csv('binary_classification_data.csv')
target = 'target_class'
y = df[target]
x = df.drop(target, axis = 1)

### 데이터 불균형 처리
- 데이터 처리

1. oversampling :

   SMOTE, ADASYN

2. undersampling

- 평가지표 처리

1. accurcacy :

   대부분의 예측이 다수 클래스에 속하게 되어 별로

2. F1-score :

   Baseline model에서 사용한 평가지표로 불균형한 데이터에서 평가지표로 사용

- 모델 구현시 처리

1. 클래스 가중치 설정 :

   소수 클래스에 더 큰 가중치 부여


In [3]:
# 유저로 부터 입력을 받아 검증 데이터 셋을 사용할 것인지, 표준화를 사용할 것인지 정함.
print("[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요")
input_1 = input("[안내] 데이터를 표준화 하시겠습니까? : ")
# 표준화 진행 여부
if input_1 == 'y':
    scaler = StandardScaler()
    X = scaler.fit_transform(x)
    print("[안내] 데이터 표준화를 진행했습니다.")
else:
    X = x
    print("[안내] 데이터 표준화를 진행하지 않습니다.")
    
input_2 = input("[안내] 검증 데이터셋을 분리할까요? : ")



# 검증 데이터 진행 여부
if input_2 == 'y':
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    print("[안내] 검증 데이터를 추가로 분리했습니다.")

else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("[안내] 검증 데이터를 분리하지 않았습니다.")



[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요
[안내] 데이터 표준화를 진행했습니다.
[안내] 검증 데이터를 추가로 분리했습니다.


In [4]:
# SMOTE로 oversampling 진행
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 클래스 가중치 계산
class_weight = {0: 1., 1: (len(y_train) / sum(y_train))}

### 모델 구현 
- 최적의 파라미터를 반복문을 통해 찾기

- 이진 분류 모델에서는 대표적으로 Adam optimizer를 사용하는 것이 일반적입니다. 다른 optimizer로는 SGD, RMSprop 등이 있으며, 이들 중에서도 데이터셋의 크기와 모델의 복잡도에 따라 적합한 optimizer를 선택해야 합니다. 하지만 대체로 Adam optimizer가 다른 optimizer보다 성능이 좋은 편입니다.

- 모델에 배치 정규화 레이어를 추가하면 모델이 더 안정적으로 수렴할 수 있습니다. 

In [ ]:
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid']
batch_lst = [8, 16, 32, 64, 128]
opt_lst = ['adam', 'rmsprop', 'sgd']
best_accuracy = 0.0
best_hyperparams = {}

# 모델 구현
for func in act_func:
    for batch in batch_lst:
        for opti in opt_lst:
            model = Sequential()
            model.add(Dense(64, activation=func, input_shape=(X_train_resampled.shape[1],)))
            model.add(Dense(32, activation=func))              
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(4, activation=func))
            model.add(Dense(1, activation='sigmoid'))

            # 모델 컴파일
            model.compile(optimizer=opti, loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=5, monitor='val_loss')

            # 모델 학습
            model.fit(X_train_resampled, y_train_resampled, epochs=1000, batch_size=batch, class_weight=class_weight, validation_data=(X_val, y_val), callbacks=[early_stopping])


            # 모델 평가
            y_pred = model.predict(X_test)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            f1 = f1_score(y_test, y_pred_binary)
            roc_auc = roc_auc_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred_binary)
            recall = recall_score(y_test, y_pred_binary)
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'batch_size': batch, 'optimizer': opti, 'dropout': 'no'}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)


In [ ]:
# dropout 추가
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid']
batch_lst = [8, 16, 32, 64, 128]
opt_lst = ['adam', 'rmsprop', 'sgd']
best_accuracy = 0.0
best_hyperparams = {}

# 모델 구현
for func in act_func:
    for batch in batch_lst:
        for opti in opt_lst:
            model = Sequential()
            model.add(Dense(64, activation=func, input_shape=(X_train_resampled.shape[1],)))
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(32, activation=func))
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(16, activation=func))
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(8, activation=func))
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(4, activation=func))
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(1, activation='sigmoid'))

            # 모델 컴파일
            model.compile(optimizer=opti, loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=5, monitor='val_loss')

            # 모델 학습
            model.fit(X_train_resampled, y_train_resampled, epochs=1000, batch_size=batch, class_weight=class_weight, validation_data=(X_val, y_val), callbacks=[early_stopping])


            # 모델 평가
            y_pred = model.predict(X_test)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            f1 = f1_score(y_test, y_pred_binary)
            roc_auc = roc_auc_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred_binary)
            recall = recall_score(y_test, y_pred_binary)
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'batch_size': batch, 'optimizer': opti, 'dropout': 'yes'}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)


In [ ]:
# dropout, 배치 정규화 추가
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid']
batch_lst = [8, 16, 32, 64, 128]
opt_lst = ['adam', 'rmsprop', 'sgd']
best_accuracy = 0.0
best_hyperparams = {}

# 모델 구현
for func in act_func:
    for batch in batch_lst:
        for opti in opt_lst:
            model = Sequential()
            model.add(Dense(64, activation=func, input_shape=(X_train_resampled.shape[1],)))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(32, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(16, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(8, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(4, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(1, activation='sigmoid'))

            # 모델 컴파일
            model.compile(optimizer=opti, loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=5, monitor='val_loss')

            # 모델 학습
            model.fit(X_train_resampled, y_train_resampled, epochs=1000, batch_size=batch, class_weight=class_weight, validation_data=(X_val, y_val), callbacks=[early_stopping])


            # 모델 평가
            y_pred = model.predict(X_test)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            f1 = f1_score(y_test, y_pred_binary)
            roc_auc = roc_auc_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred_binary)
            recall = recall_score(y_test, y_pred_binary)
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'batch_size': batch, 'optimizer': opti, 'dropout': 'yes', 'batch_normal': 'yes'}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)


### BEST MODEL

1. 구성 추가 - undersampling or oversampling 여부 

2. 최적의 파라미터 적용

3. 평가지표 어떤것을 사용할지

4. 클래스 가중치를 적용할 것인지.

### 추가사항

1. 과연 accuracy가 기존 모델보다 낮아진 모델을 사용해도 될까?: 
 accuracy를 기준으로 early stopping하면 모델의 성능은 더 증가할 것.

2. 좋은 모델의 파라미터를 사용해서 모델 구조를 변경해보기.


In [15]:
print('Best hyperparameters:', best_hyperparams)
print('Best accuracy:', best_accuracy)

Best hyperparameters: {'activation': 'tanh', 'batch_size': 64, 'optimizer': 'sgd', 'dropout': 'yes', 'batch_normal': 'yes'}
Best accuracy: 0.8460195660591125


In [ ]:
# KFold를 통해 모델 성능을 평가해보기.
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []
precision_scores = []
recall = []

for train_index, val_index in kf.split(X_train_resampled):
    X_train_fold, X_val_fold = X_train_resampled[train_index], X_train_resampled[val_index]
    y_train_fold, y_val_fold = y_train_resampled[train_index], y_train_resampled[val_index]
    
    # 모델 생성 및 학습
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train_resampled.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train_fold, y_train_fold, epochs=30, batch_size=32, class_weight=class_weight)
    
    # 검증 데이터에 대한 예측 수행
    y_pred = model.predict(X_val_fold)
    y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]
    
    # f1 score와 precision 계산
    f1_scores.append(f1_score(y_val_fold, y_pred))
    recall.append(recall_score(y_val_fold, y_pred))
    precision_scores.append(precision_score(y_val_fold, y_pred))
    
# 교차 검증 결과 출력
print('Mean F1 score:', np.mean(f1_scores))
print('Mean precision:', np.mean(precision_scores))

In [ ]:
# KFold를 통해 모델 성능을 평가해보기. - 실패
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []
precision_scores = []
recalls = []
accs = []

for train_index, val_index in kf.split(X_train_resampled):
    X_train_fold, X_val_fold = X_train_resampled[train_index], X_train_resampled[val_index]
    y_train_fold, y_val_fold = y_train_resampled[train_index], y_train_resampled[val_index]
    
    # 판단
    model = Sequential()
    model.add(Dense(64, activation='tanh', input_shape=(X_train_resampled.shape[1],)))
    model.add(BatchNormalization()) # 배치 정규화 추가
    model.add(Dropout(0.2))  # Dropout 추가
    model.add(Dense(32, activation='tanh'))
    model.add(BatchNormalization()) # 배치 정규화 추가
    model.add(Dropout(0.2))  # Dropout 추가
    model.add(Dense(1, activation='sigmoid'))

    # 모델 컴파일
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping 기능 추가
    early_stopping = EarlyStopping(patience=5, monitor='accuracy')

    model.fit(X_train_fold, y_train_fold, epochs=30, batch_size=32, class_weight=class_weight, validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])
    
    # 검증 데이터에 대한 예측 수행
    y_pred = model.predict(X_val_fold)
    y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]
    
    loss, acc = model.evaluate(X_train_fold, y_train_fold, verbose = 2)

    # f1 score와 precision 계산
    f1_scores.append(f1_score(y_val_fold, y_pred))
    recalls.append(recall_score(y_val_fold, y_pred))
    precision_scores.append(precision_score(y_val_fold, y_pred))
    accs.append(acc)
    
# 교차 검증 결과 출력
print('Mean accuracy:', np.mean(accs))
print('Mean F1 score:', np.mean(f1_scores))
print('Mean precision:', np.mean(precision_scores))
print("Mean recall:", np.mean(recalls))


In [24]:
# KFold를 통해 모델 성능을 평가해보기.
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []
precision_scores = []
recalls = []
accs = []

for train_index, val_index in kf.split(X_train_resampled):
    X_train_fold, X_val_fold = X_train_resampled[train_index], X_train_resampled[val_index]
    y_train_fold, y_val_fold = y_train_resampled[train_index], y_train_resampled[val_index]
    
    # 모델 생성 및 학습
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train_resampled.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Early stopping 기능 추가
    early_stopping = EarlyStopping(patience=5, monitor='accuracy')

    model.fit(X_train_fold, y_train_fold, epochs=1000, batch_size=32, class_weight=class_weight, validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])
    
    loss, acc = model.evaluate(X_train_fold, y_train_fold, verbose = 2)

    accs.append(acc)
    
    # 검증 데이터에 대한 예측 수행
    y_pred = model.predict(X_val_fold)
    y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]
    
    # f1 score와 precision 계산
    f1_scores.append(f1_score(y_val_fold, y_pred))
    recalls.append(recall_score(y_val_fold, y_pred))
    precision_scores.append(precision_score(y_val_fold, y_pred))
    
# 교차 검증 결과 출력
print('Mean accuracy:', np.mean(accs))
print('Mean F1 score:', np.mean(f1_scores))
print('Mean precision:', np.mean(precision_scores))
print("Mean recall:", np.mean(recalls))

Epoch 1/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.7656 - accuracy: 0.7796 - val_loss: 0.3330 - val_accuracy: 0.8354
Epoch 2/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.5263 - accuracy: 0.8474 - val_loss: 0.3037 - val_accuracy: 0.8592
Epoch 3/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.4673 - accuracy: 0.8758 - val_loss: 0.2414 - val_accuracy: 0.8993
Epoch 4/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.4410 - accuracy: 0.8860 - val_loss: 0.2719 - val_accuracy: 0.8784
Epoch 5/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.4300 - accuracy: 0.8876 - val_loss: 0.2500 - val_accuracy: 0.8916
Epoch 6/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.4113 - accuracy: 0.8950 - val_loss: 0.2088 - val_accuracy: 0.9094
Epoch 7/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.4062 - accuracy: 0.8960 - val_loss: 0.2079 - val_accuracy: